### CNN
* http://andrew.gibiansky.com/blog/machine-learning/convolutional-neural-networks/
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721
* https://algotravelling.com/ru/%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5-%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5-%D1%8D%D1%82%D0%BE-%D0%B2%D0%B5%D1%81%D0%B5%D0%BB%D0%BE-3/
* https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
* https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b
* https://www.asozykin.ru/courses/nnpython
* https://www.tensorflow.org/tutorials/deep_cnn#convolutional-neural-networks

In [1]:
from scipy import ndimage
import numpy as np
import pandas as pd
import os
from IPython.display import display, Image
from scipy import misc
import matplotlib.pyplot as plt
import tensorflow
import seaborn

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

Using TensorFlow backend.


### Keras

In [71]:
img_width = 32
img_height = 32
batch_size = 16
n_classes = 257

# faced with the issue https://stackoverflow.com/a/39851572/5151861
#                      https://github.com/fchollet/keras/issues/3945#issuecomment-274321680

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model.add(Conv2D(32, (3, 3), data_format="channels_last"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model.add(Conv2D(64, (3, 3), data_format="channels_last"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

In [96]:
model = Sequential()
model.add(Conv2D(16, (5, 5), strides=(2, 2), padding='same', kernel_initializer='he_uniform', 
           use_bias=False, input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization(scale=False))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization(scale=False))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (1, 1), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization(scale=False))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(n_classes, kernel_initializer='he_uniform'))
model.add(Activation('softmax'))

In [101]:
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [102]:
dir_train = 'data/keras/train'
dir_val = 'data/keras/validation'
dir_test = 'data/test_transformed'

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    dir_train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = val_datagen.flow_from_directory(
    dir_val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = val_datagen.flow_from_directory(
    dir_test,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 17263 images belonging to 257 classes.
Found 5854 images belonging to 257 classes.
Found 0 images belonging to 0 classes.


In [103]:
model.fit_generator(
    train_generator,
    steps_per_epoch=200,
    epochs=20,
    validation_data=test_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('first_try.h5')

Epoch 1/20
199/200 [============================>.] - ETA: 0s - loss: 5.6341 - acc: 0.0358

Exception in thread Thread-503:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/preprocessing/image.py", line 732, in __next__
    return self.next(*args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/preprocessing/image.py", line 1021, in next
    index_array, current_index, current_batch_size = next(self.index_generator)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/pytho

ValueError: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: None

In [91]:
results = model.predict_generator(val_generator, steps=150)
results = np.argmax(results, axis=1)+1

Exception in thread Thread-436:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/preprocessing/image.py", line 732, in __next__
    return self.next(*args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/preprocessing/image.py", line 1021, in next
    index_array, current_index, current_batch_size = next(self.index_generator)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/pytho

ValueError: Cannot feed value of shape () for Tensor 'conv2d_59_input:0', which has shape '(?, 32, 32, 3)'

In [ ]:
x_val = np.load('data/keras/test_images.npy')

In [ ]:
y_pred = model.predict(x_val)

In [ ]:
#results = model.predict_generator(test_generator, steps=nb_test_samples//64+1)
results = np.argmax(y_pred, axis=1)+1

In [ ]:
rs = model.predict_classes(x_val) + 1

In [ ]:
sub = pd.DataFrame(
    data=[row for row in zip(os.listdir('data/test'), results.astype(int).tolist())],
    columns=['image', 'class']
)
sub.to_csv('keras.csv', encoding='utf-8', index=False)

In [ ]:
from keras.models import model_from_json
import h5py
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
images = np.load('data/keras/train_images.npy')
targets = np.load('data/keras/targets.npy')

In [ ]:
unique_classes = np.unique(targets)
# class name -> integer
encode = {n: i for i, n in enumerate(unique_classes)}
# integer -> class name
decode = {i: n for i, n in enumerate(unique_classes)}

In [ ]:
targets = np.array([encode[n] for n in targets], dtype='int32')
targets_onehot = to_categorical(targets)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(images, targets_onehot, test_size=0.2, 
                                                    stratify=targets)
print(len(X_train), len(X_test))